In [4]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import json
import warnings
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.options.display.float_format ='{:,.4f}'.format
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format ='{:,.4f}'.format
warnings.filterwarnings('ignore')

start_date = '2024-12-01'
stop_date = '2024-12-31'

new_df = pd.read_gbq(f"""
with rides as(
select 
    city_id,
    case when order_type = 'auto_econom' then 1 
         when order_type = 'moto_econom' then 2
         when order_type = '6-seater' then 30
         when order_type = 'auto_econom_nocomment' then 31 
         when order_type = 'auto_econom_with_pet' then 34 else 0 end as order_type_id,
    count(distinct order_uuid) as cn_rides_dates
from indriver-e6e40.emart.incity_detail
where created_date_order_part between date('{start_date}') and date('{stop_date}')
  and driverdone_timestamp IS NOT NULL
group by 1,2
)
select distinct t.settings, t.city_id, t.order_type_id, t.created_date,
    r.macroregion_name, r.city_name, rides.cn_rides_dates
FROM `indriver-e6e40.ods_recprice.tariff` t
join `indriver-e6e40.heap.vw_macroregion_mapping` r
  on t.city_id = r.city_id
join rides 
    on t.city_id = rides.city_id
    and t.order_type_id = rides.order_type_id
where created_date >= '2020-01-01'
    and 
    (
    t.order_type_id in (1,2,30)
    or 
    (t.order_type_id = 34 and t.city_id = 5483)
    or 
    (t.order_type_id = 31 and t.city_id = 4244)
    )
    and t.status = 'default'
    and r.macroregion_name in ('Brazil','Latin America')
    """)

new_df['settings_list'] = new_df.apply(lambda x: {} if x['settings'] is None else json.loads(x['settings']), axis = 1)
new_df['min_price'] = new_df['settings_list'].map(lambda x: x.get('min_price'))
new_df['custom_text'] = new_df['settings_list'].map(lambda x: x.get('custom_text'))
new_df['price_round'] = new_df['settings_list'].map(lambda x: x.get('price_round'))
new_df['dynamic_round'] = new_df['settings_list'].map(lambda x: x.get('dynamic_round'))
new_df['show_block_stt'] = new_df['settings_list'].map(lambda x: x.get('show_block_stt'))
new_df['two_step_round'] = new_df['settings_list'].map(lambda x: x.get('two_step_round'))
new_df['autofill_percent'] = new_df['settings_list'].map(lambda x: x.get('autofill_percent'))
new_df['surgeon_model_id'] = new_df['settings_list'].map(lambda x: x.get('surgeon_model_id'))
new_df['rush_hour_by_surge'] = new_df['settings_list'].map(lambda x: x.get('rush_hour_by_surge'))
new_df['dynamic_min_percent'] = new_df['settings_list'].map(lambda x: x.get('dynamic_min_percent'))
new_df['dynamic_surge_enabled'] = new_df['settings_list'].map(lambda x: x.get('dynamic_surge_enabled'))
new_df['dynamic_min_ignore_surge'] = new_df['settings_list'].map(lambda x: x.get('dynamic_min_ignore_surge'))

new_df = new_df[['city_id', 'order_type_id', 'created_date', 'macroregion_name', 'city_name', 'cn_rides_dates', 'price_round', 'two_step_round']]

In [7]:
new_df[new_df['city_id'] == 4199]

,city_id,order_type_id,created_date,macroregion_name,city_name,cn_rides_dates,price_round,two_step_round
200,4199,1,2024-12-27,Latin America,Lima,12325281,0.1000,"[0.1, 1]"
452,4199,30,2025-01-20,Latin America,Lima,179086,0.1000,"[0, 0]"


In [5]:
start_date = '2025-02-01'
stop_date = '2025-02-04'
pickup_eta_minutes = 0
alpha = 0
step1 = 0.1
step2 = 1.0
city_id = 4199
type_name = 'auto_econom'

In [6]:
df_og = pd.read_gbq(f'''
                     WITH rider_data AS (SELECT city_id                     AS city_id,
                           timezone                    AS timezone,
                           type_name                   AS type_name,
                           uuid                        AS order_uuid,
                           CAST(NULL AS STRING)        AS tender_uuid,
                           'rider_price'               AS price_type,
                           payment_price_value         AS price,
                           CAST(NULL AS ARRAY <INT64>) AS available_prices,
                           price_highrate_value        AS price_highrate_value,
                           accepted_tender_uuid        AS accepted_tender_uuid,
                           AtoB_seconds                AS AtoB_seconds,
                           CAST(NULL AS INT64)         AS eta,
                           modified_at                 AS modified_at_utc,
                           order_done                  AS order_done,
                           multiplier                  AS multiplier
                    FROM `indriver-e6e40.ods_new_order_rh_cdc.order_global_strm` t1
                             LEFT JOIN (SELECT order_uuid,
                                               MAX(timezone)                                                            AS timezone,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN tender_uuid END)     AS accepted_tender_uuid,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN true ELSE false END) AS order_done,
                                               MAX(duration_in_seconds)                                                 AS AtoB_seconds
                                        FROM `indriver-e6e40.emart.incity_detail`
                                        WHERE true
                                          AND order_uuid IS NOT NULL
                                          AND created_date_order_part BETWEEN
                                            DATE_SUB('{start_date}', INTERVAL 1 DAY)
                                            AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)
                                        GROUP BY order_uuid) t2
                                       ON t1.uuid = t2.order_uuid
                    WHERE true
                      AND order_uuid IS NOT NULL
                      AND status = 'ORDER_STATUS_ACTIVE'
                      AND AtoB_seconds > 0
                      AND DATE(created_at) BETWEEN
                        DATE_SUB('{start_date}', INTERVAL 1 DAY)
                        AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)
                    QUALIFY ROW_NUMBER() OVER (PARTITION BY uuid, payment_price_value ORDER BY modified_at) = 1),

     bid_data AS (SELECT DISTINCT CAST(NULL AS INT64)                                      AS city_id,
                                  CAST(NULL AS STRING)                                     AS timezone,
                                  CAST(NULL AS STRING)                                     AS type_name,
                                  order_uuid                                               AS order_uuid,
                                  uuid                                                     AS tender_uuid,
                                  'bid_price'                                              AS price_type,
                                  price                                                    AS price,
                                  available_prices                                         AS available_prices,
                                  CAST(NULL AS INT64)                                      AS price_highrate_value,
                                  CAST(NULL AS STRING)                                     AS accepted_tender_uuid,
                                  CAST(NULL AS INT64)                                      AS AtoB_seconds,
                                  SAFE_CAST(SUBSTR(eta, 1, STRPOS(eta, 's') - 1) AS INT64) AS eta,
                                  modified_at                                              AS modified_at_utc,
                                  CAST(NULL AS BOOL)                                       AS order_done,
                                  CAST(NULL AS INT64)                                      AS multiplier
                  FROM `indriver-e6e40.ods_new_order_rh_cdc.bid_global_strm`
                  WHERE true
                    AND order_uuid IS NOT NULL
                    AND uuid IS NOT NULL
                    AND order_uuid IN (SELECT order_uuid FROM rider_data)
                    AND status = 'BID_STATUS_ACTIVE'
                    AND DATE(created_at) BETWEEN
                      DATE_SUB('{start_date}', INTERVAL 1 DAY)
                      AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)),

     my_strm AS (SELECT city_id,
                        timezone,
                        type_name,
                        order_uuid,
                        tender_uuid,
                        price_type,
                        price,
                        available_prices,
                        price_highrate_value,
                        accepted_tender_uuid,
                        AtoB_seconds,
                        eta,
                        modified_at_utc,
                        order_done,
                        multiplier
                 FROM (SELECT *
                       FROM rider_data
                       UNION ALL
                       SELECT *
                       FROM bid_data)
                 ORDER BY order_uuid, modified_at_utc),

     my_strm_full AS (SELECT LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN city_id END IGNORE NULLS)
                                        OVER w                                                           AS city_id,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN timezone END IGNORE NULLS)
                                        OVER w                                                           AS timezone,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN type_name END IGNORE NULLS)
                                        OVER w                                                           AS type_name,
                             order_uuid,
                             tender_uuid,
                             price_type,
                             price / coalesce(multiplier, 100)                                           AS price,
                             ARRAY(SELECT price / COALESCE(multiplier, 100)
                                   FROM UNNEST(available_prices) AS
                                   price)                                                                AS available_prices,
                             ARRAY_REVERSE(available_prices)[SAFE_OFFSET(0)] / COALESCE(multiplier, 100) AS last_step,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price END IGNORE
                                        NULLS)
                                        OVER w /
                             coalesce(multiplier, 100)                                                   AS start_price_value,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price_highrate_value END IGNORE
                                        NULLS)
                                        OVER w /
                             coalesce(multiplier, 100)                                                   AS price_highrate_value,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN accepted_tender_uuid END IGNORE
                                        NULLS)
                                        OVER w                                                           AS accepted_tender_uuid,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN AtoB_seconds END IGNORE
                                        NULLS)
                                        OVER w                                                           AS AtoB_seconds,
                             eta,
                             modified_at_utc,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN order_done END IGNORE
                                        NULLS) OVER w                                                    AS order_done,

                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN multiplier END IGNORE
                                        NULLS) OVER w                                                    AS multiplier
                      FROM my_strm
                      WINDOW w AS (
                              PARTITION BY order_uuid
                              ORDER BY modified_at_utc
                              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING )),

     options_assign AS (SELECT city_id,
                               type_name,
                               order_uuid,
                               tender_uuid,
                               price_type,
                               price,
                               start_price_value,
                               price_highrate_value,
                               CAST(FLOOR(AtoB_seconds / 60) * 60 AS INT64)                        AS AtoB_seconds_bin,
                               CAST(FLOOR(eta / 60) * 60 AS INT64)                                 AS eta_bin,
                               available_prices,
                               accepted_tender_uuid,
                               order_done,
                               DATETIME(TIMESTAMP(modified_at_utc), timezone)                      AS modified_at_local,
                               SAFE_DIVIDE(SAFE_DIVIDE(price, (AtoB_seconds + eta)),
                                           SAFE_DIVIDE(GREATEST(price_highrate_value, start_price_value),
                                                       (AtoB_seconds + {pickup_eta_minutes} * 60))) AS ratio,
                               last_step,
                               CASE
                                   WHEN last_step <= SAFE_DIVIDE((1 + {alpha}) *
                                                                 GREATEST(price_highrate_value, start_price_value) *
                                                                 (AtoB_seconds + eta),
                                                                 (AtoB_seconds + {pickup_eta_minutes} * 60)) THEN false
                                   ELSE
                                       true
                                   END                                                             AS new_bids_bool,
                               SAFE_DIVIDE((1 + {alpha}) *
                                           GREATEST(price_highrate_value, start_price_value) *
                                           (AtoB_seconds + eta),
                                           (AtoB_seconds + {pickup_eta_minutes} * 60))              AS max_bid,
--                                ARRAY(
--                                        SELECT start_price_value + SAFE_DIVIDE(n, ARRAY_LENGTH(available_prices)) *
--                                                                   (SAFE_DIVIDE((1 + {alpha}) *
--                                                                                GREATEST(price_highrate_value, start_price_value) *
--                                                                                (AtoB_seconds + eta),
--                                                                                (AtoB_seconds + {pickup_eta_minutes} * 60)) -
--                                                                    start_price_value)
--                                        FROM UNNEST(GENERATE_ARRAY(1, ARRAY_LENGTH(available_prices))) AS n
--                                )                                                                   AS new_bids,
--                                CAST([{step1}, {step2}] AS ARRAY <INT64>)                             AS rounds,
                               CASE
                                   WHEN last_step <= SAFE_DIVIDE((1 + {alpha}) *
                                                                 GREATEST(price_highrate_value, start_price_value) *
                                                                 (AtoB_seconds + eta),
                                                                 (AtoB_seconds + {pickup_eta_minutes} * 60))
                                       THEN available_prices
                                   ELSE ARRAY(SELECT
                                              CASE
                                                  WHEN ARRAY_LENGTH(CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)) = 2
                                                      THEN CASE
                                                               WHEN price <=
                                                                    (FLOOR(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                     CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1] +
                                                                     CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                     CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) / 2
                                                                   THEN
                                                                   CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]) *
                                                                   CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]
                                                               ELSE
                                                                   CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                   CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1] END
                                                  ELSE CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]) *
                                                       CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0] END
                                              FROM UNNEST(ARRAY(
                                                   SELECT
                                                   start_price_value + SAFE_DIVIDE(n, ARRAY_LENGTH(available_prices)) *
                                                                       (SAFE_DIVIDE((1 + {alpha}) *
                                                                                    GREATEST(price_highrate_value, start_price_value) *
                                                                                    (AtoB_seconds + eta),
                                                                                    (AtoB_seconds + {pickup_eta_minutes} * 60)) -
                                                                        start_price_value)
                                                   FROM UNNEST(GENERATE_ARRAY(1, ARRAY_LENGTH(available_prices))) AS n
                                                          )) AS price) END                         as simulated_bids
                        FROM my_strm_full
                        WHERE true
                          AND eta IS NOT NULL
                          AND available_prices IS NOT NULL
                          AND price_highrate_value IS NOT NULL

                          AND city_id = {city_id}
                          AND type_name = '{type_name}'
                          AND price_type = 'bid_price'
                          AND DATE(DATETIME(TIMESTAMP(modified_at_utc), timezone)) BETWEEN DATE('{start_date}') AND DATE('{stop_date}')
                        ORDER BY order_uuid, modified_at_local)

SELECT eta_bin,
       AtoB_seconds_bin,
       AVG(SAFE_DIVIDE(simulated_bids[ARRAY_LENGTH(simulated_bids) - 1] - start_price_value,
                       start_price_value))                                                  AS percent_range_simulated_avg,
       AVG(ARRAY_LENGTH(ARRAY(SELECT DISTINCT value FROM UNNEST(simulated_bids) AS value))) AS distinct_bids_cnt_simulated_avg,
       AVG((SELECT MIN(SAFE_DIVIDE(ABS(new_bid - price_highrate_value), price_highrate_value))
            FROM UNNEST(simulated_bids) AS new_bid))                                        AS close_bid_difference_simulated_avg,

       AVG(SAFE_DIVIDE(available_prices[ARRAY_LENGTH(available_prices) - 1] - start_price_value,
                       start_price_value))                                                  AS percent_range_avg,
       AVG(ARRAY_LENGTH(ARRAY(SELECT DISTINCT value FROM UNNEST(available_prices) AS value))) AS distinct_bids_cnt_avg,
       AVG((SELECT MIN(SAFE_DIVIDE(ABS(new_bid - price_highrate_value), price_highrate_value))
            FROM UNNEST(available_prices) AS new_bid))                                        AS close_bid_difference_avg,
FROM options_assign
GROUP BY eta_bin, AtoB_seconds_bin;

''')

In [7]:
df_og.sort_values(by='eta_bin')

,eta_bin,AtoB_seconds_bin,percent_range_simulated_avg,distinct_bids_cnt_simulated_avg,close_bid_difference_simulated_avg,percent_range_avg,distinct_bids_cnt_avg,close_bid_difference_avg
1185,-60,360,-0.1429,3.0000,0.0000,0.3571,3.0000,0.3333
3135,-60,1620,-0.0347,2.0000,0.0099,0.3119,3.0000,0.1139
800,0,780,0.1563,2.5000,0.0789,0.3024,3.0000,0.1077
3423,0,480,0.1881,3.0000,0.0071,0.3452,3.0000,0.0851
2938,0,300,0.1216,2.0000,0.0811,0.3514,3.0000,0.1486
...,...,...,...,...,...,...,...,...
132,6540,4380,0.3000,3.0000,0.0294,0.3000,3.0000,0.0294
2900,6840,180,0.3571,3.0000,0.1429,0.3571,3.0000,0.1429
1296,6900,1980,0.3182,3.0000,0.1136,0.3182,3.0000,0.1136
2230,6960,4020,0.2346,3.0000,0.1020,0.2346,3.0000,0.1020


In [8]:
df=df_og[(df_og['eta_bin'] <= 1200) & (df_og['AtoB_seconds_bin'] <= 3600)].copy()

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

for metric in [('percent_range_simulated_avg', 'percent_range_avg'),
               ('distinct_bids_cnt_simulated_avg', 'distinct_bids_cnt_avg'),
               ('close_bid_difference_simulated_avg', 'close_bid_difference_avg')]:
    # Создаем макет с 1 строкой и 2 столбцами
    fig = make_subplots(rows=1, cols=2, subplot_titles=[
        f"Heatmap of {metric[0]}",
        f"Heatmap of {metric[1]}"
    ], shared_yaxes=True)

    # Первая тепловая карта
    heatmap1 = go.Heatmap(
        x=df['eta_bin'],
        y=df['AtoB_seconds_bin'],
        z=df[metric[0]],
        colorscale='RdBu',
        zmid=0,
        colorbar=dict(title="Value", len=0.8)
    )

    # Вторая тепловая карта
    heatmap2 = go.Heatmap(
        x=df['eta_bin'],
        y=df['AtoB_seconds_bin'],
        z=df[metric[1]],
        colorscale='RdBu',
        zmid=0,
        colorbar=dict(title="Value", len=0.8, x=1.02)  # Переместим цветовую шкалу
    )

    # Добавляем графики в макет
    fig.add_trace(heatmap1, row=1, col=1)
    fig.add_trace(heatmap2, row=1, col=2)

    # Настраиваем макет
    fig.update_layout(
        title_text="Comparison",
        width=1200,
        height=600,
        template='plotly_white',
        xaxis_title="ETA (seconds)",
        yaxis_title="AtoB Seconds",
        xaxis2_title="ETA (seconds)"
    )

    # Показываем график
    fig.show()


In [10]:
df_0 = pd.read_gbq(f'''
                     WITH rider_data AS (SELECT city_id                     AS city_id,
                           timezone                    AS timezone,
                           type_name                   AS type_name,
                           uuid                        AS order_uuid,
                           CAST(NULL AS STRING)        AS tender_uuid,
                           'rider_price'               AS price_type,
                           payment_price_value         AS price,
                           CAST(NULL AS ARRAY <INT64>) AS available_prices,
                           price_highrate_value        AS price_highrate_value,
                           accepted_tender_uuid        AS accepted_tender_uuid,
                           AtoB_seconds                AS AtoB_seconds,
                           CAST(NULL AS INT64)         AS eta,
                           modified_at                 AS modified_at_utc,
                           order_done                  AS order_done,
                           multiplier                  AS multiplier
                    FROM `indriver-e6e40.ods_new_order_rh_cdc.order_global_strm` t1
                             LEFT JOIN (SELECT order_uuid,
                                               MAX(timezone)                                                            AS timezone,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN tender_uuid END)     AS accepted_tender_uuid,
                                               MAX(CASE WHEN driverdone_timestamp IS NOT NULL THEN true ELSE false END) AS order_done,
                                               MAX(duration_in_seconds)                                                 AS AtoB_seconds
                                        FROM `indriver-e6e40.emart.incity_detail`
                                        WHERE true
                                          AND order_uuid IS NOT NULL
                                          AND created_date_order_part BETWEEN
                                            DATE_SUB('{start_date}', INTERVAL 1 DAY)
                                            AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)
                                        GROUP BY order_uuid) t2
                                       ON t1.uuid = t2.order_uuid
                    WHERE true
                      AND order_uuid IS NOT NULL
                      AND status = 'ORDER_STATUS_ACTIVE'
                      AND AtoB_seconds > 0
                      AND DATE(created_at) BETWEEN
                        DATE_SUB('{start_date}', INTERVAL 1 DAY)
                        AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)
                    QUALIFY ROW_NUMBER() OVER (PARTITION BY uuid, payment_price_value ORDER BY modified_at) = 1),

     bid_data AS (SELECT DISTINCT CAST(NULL AS INT64)                                      AS city_id,
                                  CAST(NULL AS STRING)                                     AS timezone,
                                  CAST(NULL AS STRING)                                     AS type_name,
                                  order_uuid                                               AS order_uuid,
                                  uuid                                                     AS tender_uuid,
                                  'bid_price'                                              AS price_type,
                                  price                                                    AS price,
                                  available_prices                                         AS available_prices,
                                  CAST(NULL AS INT64)                                      AS price_highrate_value,
                                  CAST(NULL AS STRING)                                     AS accepted_tender_uuid,
                                  CAST(NULL AS INT64)                                      AS AtoB_seconds,
                                  SAFE_CAST(SUBSTR(eta, 1, STRPOS(eta, 's') - 1) AS INT64) AS eta,
                                  modified_at                                              AS modified_at_utc,
                                  CAST(NULL AS BOOL)                                       AS order_done,
                                  CAST(NULL AS INT64)                                      AS multiplier
                  FROM `indriver-e6e40.ods_new_order_rh_cdc.bid_global_strm`
                  WHERE true
                    AND order_uuid IS NOT NULL
                    AND uuid IS NOT NULL
                    AND order_uuid IN (SELECT order_uuid FROM rider_data)
                    AND status = 'BID_STATUS_ACTIVE'
                    AND DATE(created_at) BETWEEN
                      DATE_SUB('{start_date}', INTERVAL 1 DAY)
                      AND DATE_ADD('{stop_date}', INTERVAL 1 DAY)),

     my_strm AS (SELECT city_id,
                        timezone,
                        type_name,
                        order_uuid,
                        tender_uuid,
                        price_type,
                        price,
                        available_prices,
                        price_highrate_value,
                        accepted_tender_uuid,
                        AtoB_seconds,
                        eta,
                        modified_at_utc,
                        order_done,
                        multiplier
                 FROM (SELECT *
                       FROM rider_data
                       UNION ALL
                       SELECT *
                       FROM bid_data)
                 ORDER BY order_uuid, modified_at_utc),

     my_strm_full AS (SELECT LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN city_id END IGNORE NULLS)
                                        OVER w                                                           AS city_id,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN timezone END IGNORE NULLS)
                                        OVER w                                                           AS timezone,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN type_name END IGNORE NULLS)
                                        OVER w                                                           AS type_name,
                             order_uuid,
                             tender_uuid,
                             price_type,
                             price / coalesce(multiplier, 100)                                           AS price,
                             ARRAY(SELECT price / COALESCE(multiplier, 100)
                                   FROM UNNEST(available_prices) AS
                                   price)                                                                AS available_prices,
                             ARRAY_REVERSE(available_prices)[SAFE_OFFSET(0)] / COALESCE(multiplier, 100) AS last_step,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price END IGNORE
                                        NULLS)
                                        OVER w /
                             coalesce(multiplier, 100)                                                   AS start_price_value,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN price_highrate_value END IGNORE
                                        NULLS)
                                        OVER w /
                             coalesce(multiplier, 100)                                                   AS price_highrate_value,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN accepted_tender_uuid END IGNORE
                                        NULLS)
                                        OVER w                                                           AS accepted_tender_uuid,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN AtoB_seconds END IGNORE
                                        NULLS)
                                        OVER w                                                           AS AtoB_seconds,
                             eta,
                             modified_at_utc,
                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN order_done END IGNORE
                                        NULLS) OVER w                                                    AS order_done,

                             LAST_VALUE(CASE WHEN price_type = 'rider_price' THEN multiplier END IGNORE
                                        NULLS) OVER w                                                    AS multiplier
                      FROM my_strm
                      WINDOW w AS (
                              PARTITION BY order_uuid
                              ORDER BY modified_at_utc
                              ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING )),

     options_assign AS (SELECT city_id,
                               type_name,
                               order_uuid,
                               tender_uuid,
                               price_type,
                               price,
                               start_price_value,
                               price_highrate_value,
                               CAST(FLOOR(AtoB_seconds / 60) * 60 AS INT64)                        AS AtoB_seconds_bin,
                               CAST(FLOOR(eta / 60) * 60 AS INT64)                                 AS eta_bin,
                               available_prices,
                               accepted_tender_uuid,
                               order_done,
                               DATETIME(TIMESTAMP(modified_at_utc), timezone)                      AS modified_at_local,
                               SAFE_DIVIDE(SAFE_DIVIDE(price, (AtoB_seconds + eta)),
                                           SAFE_DIVIDE(GREATEST(price_highrate_value, start_price_value),
                                                       (AtoB_seconds + {pickup_eta_minutes} * 60))) AS ratio,
                               last_step,
                               CASE
                                   WHEN last_step <= SAFE_DIVIDE((1 + {alpha}) *
                                                                 GREATEST(price_highrate_value, start_price_value) *
                                                                 (AtoB_seconds + eta),
                                                                 (AtoB_seconds + {pickup_eta_minutes} * 60)) THEN false
                                   ELSE
                                       true
                                   END                                                             AS new_bids_bool,
                               SAFE_DIVIDE((1 + {alpha}) *
                                           GREATEST(price_highrate_value, start_price_value) *
                                           (AtoB_seconds + eta),
                                           (AtoB_seconds + {pickup_eta_minutes} * 60))              AS max_bid,
--                                ARRAY(
--                                        SELECT start_price_value + SAFE_DIVIDE(n, ARRAY_LENGTH(available_prices)) *
--                                                                   (SAFE_DIVIDE((1 + {alpha}) *
--                                                                                GREATEST(price_highrate_value, start_price_value) *
--                                                                                (AtoB_seconds + eta),
--                                                                                (AtoB_seconds + {pickup_eta_minutes} * 60)) -
--                                                                    start_price_value)
--                                        FROM UNNEST(GENERATE_ARRAY(1, ARRAY_LENGTH(available_prices))) AS n
--                                )                                                                   AS new_bids,
--                                CAST([{step1}, {step2}] AS ARRAY <INT64>)                             AS rounds,
                               CASE
                                   WHEN last_step <= SAFE_DIVIDE((1 + {alpha}) *
                                                                 GREATEST(price_highrate_value, start_price_value) *
                                                                 (AtoB_seconds + eta),
                                                                 (AtoB_seconds + {pickup_eta_minutes} * 60))
                                       THEN available_prices
                                   ELSE ARRAY(SELECT
                                              CASE
                                                  WHEN ARRAY_LENGTH(CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)) = 2
                                                      THEN CASE
                                                               WHEN price <=
                                                                    (FLOOR(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                     CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1] +
                                                                     CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                     CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) / 2
                                                                   THEN
                                                                   CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]) *
                                                                   CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]
                                                               ELSE
                                                                   CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1]) *
                                                                   CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[1] END
                                                  ELSE CEIL(price / CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0]) *
                                                       CAST([{step1}, {step2}] AS ARRAY <FLOAT64>)[0] END
                                              FROM UNNEST(ARRAY(
                                                   SELECT
                                                   start_price_value + SAFE_DIVIDE(n, ARRAY_LENGTH(available_prices)) *
                                                                       (SAFE_DIVIDE((1 + {alpha}) *
                                                                                    GREATEST(price_highrate_value, start_price_value) *
                                                                                    (AtoB_seconds + eta),
                                                                                    (AtoB_seconds + {pickup_eta_minutes} * 60)) -
                                                                        start_price_value)
                                                   FROM UNNEST(GENERATE_ARRAY(1, ARRAY_LENGTH(available_prices))) AS n
                                                          )) AS price) END                         as simulated_bids
                        FROM my_strm_full
                        WHERE true
                          AND eta IS NOT NULL
                          AND available_prices IS NOT NULL
                          AND price_highrate_value IS NOT NULL

                          AND city_id = {city_id}
                          AND type_name = '{type_name}'
                          AND price_type = 'bid_price'
                          AND DATE(DATETIME(TIMESTAMP(modified_at_utc), timezone)) BETWEEN DATE('{start_date}') AND DATE('{stop_date}')
                        ORDER BY order_uuid, modified_at_local)

SELECT AVG(SAFE_DIVIDE(simulated_bids[ARRAY_LENGTH(simulated_bids) - 1] - start_price_value,
                       start_price_value))                                                  AS percent_range_simulated_avg,
       AVG(ARRAY_LENGTH(ARRAY(SELECT DISTINCT value FROM UNNEST(simulated_bids) AS value))) AS distinct_bids_cnt_simulated_avg,
       AVG((SELECT MIN(SAFE_DIVIDE(ABS(new_bid - price_highrate_value), price_highrate_value))
            FROM UNNEST(simulated_bids) AS new_bid))                                        AS close_bid_difference_simulated_avg,

       AVG(SAFE_DIVIDE(available_prices[ARRAY_LENGTH(available_prices) - 1] - start_price_value,
                       start_price_value))                                                  AS percent_range_avg,
       AVG(ARRAY_LENGTH(ARRAY(SELECT DISTINCT value FROM UNNEST(available_prices) AS value))) AS distinct_bids_cnt_avg,
       AVG((SELECT MIN(SAFE_DIVIDE(ABS(new_bid - price_highrate_value), price_highrate_value))
            FROM UNNEST(available_prices) AS new_bid))                                        AS close_bid_difference_avg,
FROM options_assign;
''')

df_0


,percent_range_simulated_avg,distinct_bids_cnt_simulated_avg,close_bid_difference_simulated_avg,percent_range_avg,distinct_bids_cnt_avg,close_bid_difference_avg
0,0.2213,2.9645,0.0875,0.3060,3.0000,0.1140


In [11]:
df_0

,percent_range_simulated_avg,distinct_bids_cnt_simulated_avg,close_bid_difference_simulated_avg,percent_range_avg,distinct_bids_cnt_avg,close_bid_difference_avg
0,0.2213,2.9645,0.0875,0.3060,3.0000,0.1140
